In [1]:
pip install pandas scikit-learn ipywidgets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 11.2 MB/s eta 0:00:00


In [4]:
import pandas as pd

# Load the dataset
df = pd.read_csv('/content/drive/MyDrive/ipl_colab.csv')

# Group by match ID and aggregate data
match_data = df.groupby('mid').agg({
    'date': 'first',  # First occurrence of the date
    'venue': 'first',  # First occurrence of the venue
    'batting_team': 'first',  # First team to bat
    'bowling_team': 'first',  # First team to bowl
    'total': 'max',  # Total runs in the match
    'runs_last_5': 'sum',  # Sum of runs in the last 5 overs
    'wickets_last_5': 'sum',  # Sum of wickets in the last 5 overs
}).reset_index()

# Rename columns for clarity
match_data.rename(columns={
    'batting_team': 'team1',
    'bowling_team': 'team2',
    'total': 'total_runs',
    'runs_last_5': 'runs_last_5_total',
    'wickets_last_5': 'wickets_last_5_total',
}, inplace=True)

# Add a 'winner' column (you need to define this based on your dataset)
# For example, if the team with the highest total runs wins:
match_data['winner'] = match_data.apply(lambda row: row['team1'] if row['total_runs'] > row['total_runs'] else row['team2'], axis=1)

# Display the match-level data
print(match_data.head())

   mid        date                                       venue  \
0    1  2008-04-18                       M Chinnaswamy Stadium   
1    2  2008-04-19  Punjab Cricket Association Stadium, Mohali   
2    3  2008-04-19                            Feroz Shah Kotla   
3    4  2008-04-20                            Wankhede Stadium   
4    5  2008-04-20                                Eden Gardens   

                   team1                        team2  total_runs  \
0  Kolkata Knight Riders  Royal Challengers Bangalore         222   
1    Chennai Super Kings              Kings XI Punjab         240   
2       Rajasthan Royals             Delhi Daredevils         129   
3         Mumbai Indians  Royal Challengers Bangalore         165   
4        Deccan Chargers        Kolkata Knight Riders         110   

   runs_last_5_total  wickets_last_5_total                       winner  
0               5416                    78  Royal Challengers Bangalore  
1               5852                   1

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import joblib

# Encode categorical variables
label_encoders = {}
for column in ['team1', 'team2', 'venue', 'winner']:
    le = LabelEncoder()
    match_data[column] = le.fit_transform(match_data[column])
    label_encoders[column] = le

# Define features and target variable
X = match_data[['team1', 'team2', 'venue', 'total_runs', 'runs_last_5_total', 'wickets_last_5_total']]
y = match_data['winner']

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Train a RandomForestClassifier
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)

# Save the model and preprocessing objects
joblib.dump(rf, 'ipl_winner_predictor.pkl')
joblib.dump(scaler, 'scaler.pkl')
joblib.dump(label_encoders, 'label_encoders.pkl')

['label_encoders.pkl']

In [6]:
import ipywidgets as widgets
from IPython.display import display
import joblib

# Load the saved model and preprocessing objects
rf = joblib.load('ipl_winner_predictor.pkl')
scaler = joblib.load('scaler.pkl')
label_encoders = joblib.load('label_encoders.pkl')

# Create dropdowns for user input
team1_dropdown = widgets.Dropdown(
    options=label_encoders['team1'].classes_,
    description='Team 1:'
)

team2_dropdown = widgets.Dropdown(
    options=label_encoders['team2'].classes_,
    description='Team 2:'
)

venue_dropdown = widgets.Dropdown(
    options=label_encoders['venue'].classes_,
    description='Venue:'
)

# Button to trigger prediction
predict_button = widgets.Button(description="Predict Winner")

# Output widget to display the result
output = widgets.Output()

# Function to handle button click
def on_predict_button_click(b):
    # Get user inputs
    team1 = team1_dropdown.value
    team2 = team2_dropdown.value
    venue = venue_dropdown.value

    # Encode inputs using label encoders
    team1_encoded = label_encoders['team1'].transform([team1])[0]
    team2_encoded = label_encoders['team2'].transform([team2])[0]
    venue_encoded = label_encoders['venue'].transform([venue])[0]

    # Create feature vector
    features = [
        team1_encoded,
        team2_encoded,
        venue_encoded,
        160,  # Example: total_runs (replace with actual value)
        40,   # Example: runs_last_5_total (replace with actual value)
        2     # Example: wickets_last_5_total (replace with actual value)
    ]

    # Scale the features
    features_scaled = scaler.transform([features])

    # Make prediction
    prediction_encoded = rf.predict(features_scaled)[0]
    prediction = label_encoders['winner'].inverse_transform([prediction_encoded])[0]

    # Display the result
    with output:
        output.clear_output()
        print(f"Predicted Winner: {prediction}")

# Attach the function to the button
predict_button.on_click(on_predict_button_click)

# Display the widgets
display(team1_dropdown, team2_dropdown, venue_dropdown, predict_button, output)

Dropdown(description='Team 1:', options=('Chennai Super Kings', 'Deccan Chargers', 'Delhi Daredevils', 'Gujara…

Dropdown(description='Team 2:', options=('Chennai Super Kings', 'Deccan Chargers', 'Delhi Daredevils', 'Gujara…

Dropdown(description='Venue:', options=('Barabati Stadium', 'Brabourne Stadium', 'Buffalo Park', 'De Beers Dia…

Button(description='Predict Winner', style=ButtonStyle())

Output()

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
